In [44]:
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import *
from keras.applications import VGG16

import numpy as np
import matplotlib.pyplot as plt
import cv2

# 1.DATA

Uploading data I prepared in tfrecord format. Both image pixels and box points are normalized. 

Format of box points: [x_min, y_min, x_max, y_max]

In [45]:
train = tf.data.Dataset.load(os.path.join('train.tfrecord')).shuffle(5000).batch(8).prefetch(tf.data.AUTOTUNE)
test = tf.data.Dataset.load(os.path.join('test.tfrecord')).shuffle(2000).batch(8).prefetch(tf.data.AUTOTUNE)

In [47]:
train, test

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 120, 120, None), dtype=tf.float32, name=None), (TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None), TensorSpec(shape=<unknown>, dtype=tf.float16, name=None)))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 120, 120, None), dtype=tf.float32, name=None), (TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None), TensorSpec(shape=<unknown>, dtype=tf.float16, name=None)))>)

In [48]:
len(train), len(test)

(524, 125)

# 2.MODEL

### Optimizer

In [49]:
batches = len(train)
lr_decay = (1./0.75 -1)/batches

opt = tf.keras.optimizers.legacy.Adam(0.0001, decay=lr_decay)

### Loss Functions

  Smooth L1 Loss Function is commonly used for calculating localization loss.



In [50]:
def smooth_l1_loss(y_true, y_pred):
    """
    Smooth L1 loss for bounding box regression.
    y_true: True bounding box coordinates (x_min, y_min, x_max, y_max)
    y_pred: Predicted bounding box coordinates (x_min, y_min, x_max, y_max)
    """
    diff = y_true - y_pred
    abs_diff = tf.abs(diff)
    squared_loss = 0.5 * (diff ** 2)
    linear_loss = abs_diff - 0.5
    loss = tf.where(abs_diff < 1.0, squared_loss, linear_loss)
    return tf.reduce_mean(loss)

class_loss = tf.keras.losses.BinaryCrossentropy()

### Creating SSD Custom Model

SSD consist of 3 parts:


1.   VGG16: For feature extraction.
2.   Classification: For determining whether there is a face in image or not.
3.   Localization: With regression, tries to predict the position coordinates of the box around the face.










In [51]:
class Face_Detector(Model):
  def __init__(self, input_shape):
    super(SSD, self).__init__()
    self.model = self.build_model(input_shape)


  # Builds main SSD model
  def build_model(self, input_shape):
    input = Input(shape=input_shape)
    vgg = VGG16(include_top=False, input_shape=input_shape)(input)

    x = GlobalMaxPooling2D()(vgg)
    x = Dense(2048, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)

    y = GlobalMaxPooling2D()(vgg)
    y = Dense(2048, activation='relu')(y)
    y = Dense(4, activation='sigmoid')(y)

    return Model(inputs=input, outputs=[x,y])

  # Stores the optimizer, classification loss and localization loss functions
  def compile(self, opt, class_loss, box_loss):
    super().compile()
    self.opt = opt
    self.closs_func = class_loss
    self.bloss_func = box_loss

  # Custom training step
  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      label, coor = self.model(x, training=True)

      # Compute the losses
      class_loss_val = self.closs_func(y[0], label)
      box_loss_val = self.bloss_func(tf.cast(y[1], tf.float32), coor)

      # Combine the losses using weighted sum or any other appropriate combination
      total_loss = class_loss_val + 10*box_loss_val

      gradients = tape.gradient(total_loss, self.model.trainable_variables)

    # Update model weights
    self.opt.apply_gradients(zip(gradients, self.model.trainable_variables))

    return {"total_loss":total_loss, "class_loss":class_loss_val, "boxing_loss":box_loss_val}


  # Custom test step
  def test_step(self, data):
    x, y = data
    label, coor = self.model(x, training=False)

    # Compute the losses
    class_loss_val = self.closs_func(y[0], label)
    box_loss_val = self.bloss_func(tf.cast(y[1], tf.float32), coor)

    # Combine the losses using weighted sum or any other appropriate combination
    total_loss = class_loss_val + 10*box_loss_val

    return {"total_loss":total_loss, "class_loss":class_loss_val, "boxing_loss":box_loss_val}


  def call(self, inputs):
    return self.model(inputs)


### Building the Model

In [52]:
input_shape=(120, 120, 3)
model = Face_Detector(input_shape)

model.compile(opt, class_loss, smooth_l1_loss)

### Training the Model

In [53]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_total_loss', patience=5, restore_best_weights=True)
model.fit(train, epochs=30, validation_data=test, callbacks=[early_stopping])

Epoch 1/40
524/524 [==============================] - 36s 64ms/step - total_loss: 0.1330 - class_loss: 0.0655 - boxing_loss: 0.0067 - val_total_loss: 0.0106 - val_class_loss: 5.9605e-07 - val_boxing_loss: 0.0011
Epoch 2/40
524/524 [==============================] - 33s 61ms/step - total_loss: 0.0397 - class_loss: 0.0217 - boxing_loss: 0.0018 - val_total_loss: 0.0055 - val_class_loss: 0.0017 - val_boxing_loss: 3.8124e-04
Epoch 3/40
524/524 [==============================] - 33s 62ms/step - total_loss: 0.0122 - class_loss: 0.0056 - boxing_loss: 6.5416e-04 - val_total_loss: 0.0017 - val_class_loss: 0.0000e+00 - val_boxing_loss: 1.6929e-04
Epoch 4/40
524/524 [==============================] - 33s 63ms/step - total_loss: 0.0098 - class_loss: 0.0042 - boxing_loss: 5.5714e-04 - val_total_loss: 0.0047 - val_class_loss: 2.9923e-05 - val_boxing_loss: 4.7189e-04
Epoch 5/40
524/524 [==============================] - 33s 62ms/step - total_loss: 0.0078 - class_loss: 0.0034 - boxing_loss: 4.4653e-04 

### Testing the Model

In [54]:
test_data = test.as_numpy_iterator()

test_sample = test_data.next()
preds = model.predict(test_sample[0])

1/1 [==============================] - 0s 152ms/step


In [ ]:
image = test_sample[0][0]
coords = preds[1][0]

if preds[0][0] > 0.9:
  # Drawing the box according to predicted coordinates
  cv2.rectangle(image,
                    tuple(np.multiply(coords[:2], [120,120]).astype(int)),
                    tuple(np.multiply(coords[2:], [120,120]).astype(int)),
                          (255,0,0), 2)

plt.imshow(image)

### Saving the Model

In [56]:
tf.saved_model.save(model, 'Face_Detector_Model')